In [1]:
from __future__ import division
import networkx as nx
import matplotlib.pyplot as plt
import pylab
import scipy as sp
import numpy as np
import itertools
import copy
import math
import random

N =  50                  # Number of nodes
rw = 700                 # Wealth value of rich
pw = 300                # Wealth value of poor
we_arr = [rw, pw]       # Array storing both the wealth values
pr = 0.5                # Probability of being rich
prob_arr = [pr, 1-pr]  # Array storing probability of wealth allotment values
M = 20                  # Number of rounds
co = 50                 # Unitary Cost of co-operation
be = 100                # Unitary Benefit of co-operation
S = 100                  # Number of iterations
pl = 0.3                # Initial probability of a link being formed between any two nodes

p0_arr = [0.55,0.7,0.9]
re_arr = [0,0.3,0.6]

def inv_logit(y):
    inv_logit_out =(np.exp(y)/(1+np.exp(y)))
    return inv_logit_out

def gini_calc(x):       #IN: Wealth values across the population, OUT: Gini Coefficient
    sum2=0
    for i in range(len(x)):
        sum1=0
        for j in range(len(x)):
            sum1=sum1+np.abs(x[i]-x[j])
        sum2=sum2+sum1;
    
    gini=sum2/(2*len(x)**2*np.mean(x));
    return gini

node_arr = np.arange(N)

c_ar = np.zeros([len(p0_arr),len(re_arr),S,M])     # Array storing number of cooperative environments
d_ar = np.zeros([len(p0_arr),len(re_arr),S,M])     # Array storing number of selfish environments
eq_ar = np.zeros([len(p0_arr),len(re_arr),S,M])   # Array storing number of neutral environments

flag = 0

for h1 in range(len(re_arr)):    
    re = re_arr[h1]
    for h2 in range(len(p0_arr)):
        p0 = p0_arr[h2]
        for h in range(S):
            # Initializing the counts
            c_env = 0
            d_env = 0
            eq_env = 0

            G = nx.erdos_renyi_graph(N, pl)
            A = nx.adjacency_matrix(G)
            E = G.number_of_edges()
            No_of_pairs = (N*(N-1))/2
            No_of_rewiring = round(No_of_pairs*re)
            Nre = int(No_of_rewiring)

            init_wealth_array = np.random.choice(we_arr,N,p=prob_arr)             

            for k in range(M):                 # Continuing the game for M rounds
                c_env = 0
                d_env = 0
                eq_env = 0

                if (k==0):                    # First round 
                    p_in = 0.7
                    decision_array = np.zeros(N)
                    for i in range(N):         # For all nodes in the network
                        nn = G.degree(i)
                        a = np.random.random()

                        if (a <= p_in):
                            decision_array[i] = 1       # Node-i decides to be cooperative                    
                            init_wealth_array[i] = init_wealth_array[i] - co*nn   # Updatation of wealth of a cooperative node
                            for l in range(N):          # Updatation of wealth of its neighbouring nodes
                                if (i==l):
                                    continue
                                else:
                                    if (A[i,l] == 1):
                                        init_wealth_array[l] = init_wealth_array[l] + be
                                    else:
                                        continue

                        else:
                            decision_array[i] = 0          # Node-i decides to be non-cooperative
                            continue                  


                else:                     # Decision making in subsequent rounds
                    new_decision_array = np.zeros(N)

                    for i in range(N):         # For all nodes in the network
                        nn = G.degree(i)                                    
                        cop = 0                     # Initializing the no of cooperator neighbours
                        for j in range(N):
                            if (A[i,j] == 1):               # If a link exists between nodes i and j
                                if (decision_array[j] == 1):        # If link j cooperated in last round
                                    cop += 1
                                elif (decision_array[j] == 0):      # If link j did not cooperate in last round
                                    continue
                            else:
                                continue

                        sum_neighbor_wealth = 0
                        for j in range(N):			# Total wealth of the neighbors
                            if (A[i,j] == 1):
                                sum_neighbor_wealth += init_wealth_array[j]
                            else:
                                continue

                        if (nn>0):
                            av_neighbor_wealth = (sum_neighbor_wealth)/nn
                            diff_wealth = av_neighbor_wealth - (init_wealth_array[i])
                            pc = p0 + (1-p0)*(np.tanh(0.001*diff_wealth)) - 0.10 
                        else:
                            pc = p0                   


                        if (nn>0):              # For postive number of nearest neighbours
                            cop_rat = float(cop/nn)                    
                            if (cop_rat > 0.5):       # If cooperation parameter is greater than 0.5 i.e. cooperative environment

                                c = np.random.random()
                                if (c < pc):
                                    new_decision_array[i] = 1       # Node-i decides to be cooperative                          
                                else:
                                    new_decision_array[i] = 0

                            elif (cop_rat < 0.5):          # Cooperation parameter is lesser than 0.5 i.e. defective environment

                                c = np.random.random()
                                if (c < pc):                        
                                    new_decision_array[i] = 0
                                else:
                                    new_decision_array[i] = 1

                            elif (cop_rat == 0.5):       # Neutral environment

                                  c = np.random.random()
                                  if (c<=pc):
                                      new_decision_array[i] = 1
                                  else:
                                      new_decision_array[i] = 0

                        else:
                            new_decision_array[i] = 0

                    for i in range(N):
                        if (new_decision_array[i] == 1): 
                            init_wealth_array[i] = init_wealth_array[i] - co*(G.degree(i))   # Updatation of wealth of a cooperative node
                            for l in range(N):          	# Updatation of wealth of its neighbouring nodes
                                if (i==l):
                                    continue
                                else:
                                    if (A[i,l] == 1):
                                        init_wealth_array[l] = init_wealth_array[l] + be
                                    else:
                                        continue
                        else:
                            continue       


                    decision_array = copy.copy(new_decision_array)         

                # Rewiring step

                temp_array = []


                while (len(temp_array) < Nre) :            
                    i = random.choice(node_arr)
                    j = random.choice(node_arr)

                    if (i==j):
                        continue
                    else:

                        if (i,j) in temp_array:
                            continue
                        elif (j,i) in temp_array:
                            continue
                        else:
                            temp_array.append((i,j))
                            temp = [i,j]

                            if (A[i,j] == 1):                              # If a link exists between the selected pair
                                b = random.choice(temp)

                                if (b == i):                               # If node-i is given the choice to break or not
                                    if (decision_array[j] == 0):
                                        c = np.random.random()
                                        if (c < 0.7):
                                            G.remove_edge(i,j)                 # Breakage of link for node-i disagreeing

                                        else:
                                            continue                        # Link is kept intact
                                    else:
                                        c = np.random.random()
                                        if (c < 0.87):
                                            continue
                                        else:
                                            G.remove_edge(i,j)
                                else:                                   # If node-j is given the choice to break or not	
                                    if (decision_array[i] == 0):
                                        c = np.random.random()
                                        if (c < 0.7):                                    
                                            G.remove_edge(i,j)         # Breakage of link for node-j disagreeing
                                        else:
                                            continue			# Link is kept intact
                                    else:
                                        c = np.random.random()
                                        if (c < 0.87):
                                            continue
                                        else:
                                            G.remove_edge(i,j)


                            else:                 # If a link does not exist between the selected pair
                                if (decision_array[i] == 1 and decision_array[j] == 1):
                                    c = np.random.random()
                                    if (c < 0.93):
                                        G.add_edge(i,j)
                                    else:
                                        continue

                                if (decision_array[i] == 0 and decision_array[j] == 0):
                                    c = np.random.random()
                                    if (c < 0.8):
                                        continue
                                    else:
                                        G.add_edge(i,j)

                                if (decision_array[i] == 0 and decision_array[j] == 1):
                                    c = np.random.random()
                                    if (c < 0.7):
                                        continue
                                    else:
                                        G.add_edge(i,j)

                                if (decision_array[i] == 1 and decision_array[j] == 0):
                                    c = np.random.random()
                                    if (c < 0.7):
                                        continue
                                    else:
                                        G.add_edge(i,j)

                A = nx.adjacency_matrix(G)     # Adjacency matrix of the network

                for i in range(N):         # For all nodes in the network                                                   
                    cop = 0                     # Initializing the no of cooperator neighbours
                    for j in range(N):
                        if (A[i,j] == 1):               # If a link exists between nodes i and j
                            if (decision_array[j] == 1):        # If link j cooperated in last round
                                cop += 1
                            elif (decision_array[j] == 0):      # If link j did not cooperate in last round
                                continue
                        else:
                            continue
                    if (G.degree(i)>0):
                        cop_rat = float(cop/G.degree(i))                    
                        if (cop_rat > 0.5):       # If cooperation parameter is greater than 0.5
                                c_env += 1
                        elif (cop_rat == 0.5):     # Neutral environment
                                eq_env += 1
                        elif(cop_rat < 0.5):       # Selfish environment
                                d_env += 1
                    else:
                            d_env += 1      # A node without any bonds is assumed to be in a selfish environment

                c_ar[h2,h1,h,k] = c_env
                d_ar[h2,h1,h,k] = d_env
                eq_ar[h2,h1,h,k] = eq_env

            flag += 1
            yo = (flag*100)/(len(re_arr)*S*len(p0_arr))

            print('Job completion percentage: %f' %yo)

np.savez('Supl-4', c_ar=c_ar,d_ar=d_ar,eq_ar=eq_ar)   


Job completion percentage: 0.111111
Job completion percentage: 0.222222
Job completion percentage: 0.333333
Job completion percentage: 0.444444
Job completion percentage: 0.555556
Job completion percentage: 0.666667
Job completion percentage: 0.777778
Job completion percentage: 0.888889
Job completion percentage: 1.000000
Job completion percentage: 1.111111
Job completion percentage: 1.222222
Job completion percentage: 1.333333
Job completion percentage: 1.444444
Job completion percentage: 1.555556
Job completion percentage: 1.666667
Job completion percentage: 1.777778
Job completion percentage: 1.888889
Job completion percentage: 2.000000
Job completion percentage: 2.111111
Job completion percentage: 2.222222
Job completion percentage: 2.333333
Job completion percentage: 2.444444
Job completion percentage: 2.555556
Job completion percentage: 2.666667
Job completion percentage: 2.777778
Job completion percentage: 2.888889
Job completion percentage: 3.000000
Job completion percentage: 3

Job completion percentage: 25.000000
Job completion percentage: 25.111111
Job completion percentage: 25.222222
Job completion percentage: 25.333333
Job completion percentage: 25.444444
Job completion percentage: 25.555556
Job completion percentage: 25.666667
Job completion percentage: 25.777778
Job completion percentage: 25.888889
Job completion percentage: 26.000000
Job completion percentage: 26.111111
Job completion percentage: 26.222222
Job completion percentage: 26.333333
Job completion percentage: 26.444444
Job completion percentage: 26.555556
Job completion percentage: 26.666667
Job completion percentage: 26.777778
Job completion percentage: 26.888889
Job completion percentage: 27.000000
Job completion percentage: 27.111111
Job completion percentage: 27.222222
Job completion percentage: 27.333333
Job completion percentage: 27.444444
Job completion percentage: 27.555556
Job completion percentage: 27.666667
Job completion percentage: 27.777778
Job completion percentage: 27.888889
J

Job completion percentage: 49.666667
Job completion percentage: 49.777778
Job completion percentage: 49.888889
Job completion percentage: 50.000000
Job completion percentage: 50.111111
Job completion percentage: 50.222222
Job completion percentage: 50.333333
Job completion percentage: 50.444444
Job completion percentage: 50.555556
Job completion percentage: 50.666667
Job completion percentage: 50.777778
Job completion percentage: 50.888889
Job completion percentage: 51.000000
Job completion percentage: 51.111111
Job completion percentage: 51.222222
Job completion percentage: 51.333333
Job completion percentage: 51.444444
Job completion percentage: 51.555556
Job completion percentage: 51.666667
Job completion percentage: 51.777778
Job completion percentage: 51.888889
Job completion percentage: 52.000000
Job completion percentage: 52.111111
Job completion percentage: 52.222222
Job completion percentage: 52.333333
Job completion percentage: 52.444444
Job completion percentage: 52.555556
J

Job completion percentage: 74.333333
Job completion percentage: 74.444444
Job completion percentage: 74.555556
Job completion percentage: 74.666667
Job completion percentage: 74.777778
Job completion percentage: 74.888889
Job completion percentage: 75.000000
Job completion percentage: 75.111111
Job completion percentage: 75.222222
Job completion percentage: 75.333333
Job completion percentage: 75.444444
Job completion percentage: 75.555556
Job completion percentage: 75.666667
Job completion percentage: 75.777778
Job completion percentage: 75.888889
Job completion percentage: 76.000000
Job completion percentage: 76.111111
Job completion percentage: 76.222222
Job completion percentage: 76.333333
Job completion percentage: 76.444444
Job completion percentage: 76.555556
Job completion percentage: 76.666667
Job completion percentage: 76.777778
Job completion percentage: 76.888889
Job completion percentage: 77.000000
Job completion percentage: 77.111111
Job completion percentage: 77.222222
J

Job completion percentage: 99.000000
Job completion percentage: 99.111111
Job completion percentage: 99.222222
Job completion percentage: 99.333333
Job completion percentage: 99.444444
Job completion percentage: 99.555556
Job completion percentage: 99.666667
Job completion percentage: 99.777778
Job completion percentage: 99.888889
Job completion percentage: 100.000000
